### Постановка задачи:  
Разработать модель предсказания в каждом регионе объема продаж в штуках любого, выбранного вами, вида банки напитка Coca Cola.  
Модель должна предсказывать объем продаж на каждый час следующих суток (от последней даты транзакции в датасете).

In [65]:
import pandas as pd
import numpy as np
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.arima import AutoARIMA
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")

In [66]:
data = pd.read_csv('dataset.csv', index_col=0)
display(data.sample(10, random_state=13))
print('Размерность датасета: ', data.shape)

,item_id,receipt_id,device_id,local_date,barcode,price,quantity,region
475282,11725120973,7401055919,352401547525226,2021-02-06 13:10:40,4600084388665,24.000000,1.000,Пермский край
2102122,11682540295,7377118917,352407076335888,2021-02-03 13:20:17,2000000000107,246.990005,0.442,Краснодарский край
1531729,11734646590,7406344901,352407930590845,2021-02-06 20:50:06,4606419005870,130.000000,1.000,Московская область
757038,11710050361,7392724566,352406575284803,2021-02-05 11:43:54,NaN,210.000000,1.000,Ростовская область
1147232,11732205129,7404993403,352403527442486,2021-02-06 16:50:32,NaN,5.000000,1.000,Новгородская область
686105,11726121192,7401605407,352406278254393,2021-02-06 11:53:06,4810344073020,1000.000000,1.000,Курская область
1574081,11667479891,7368510518,352406834320179,2021-02-02 12:47:07,NaN,90.000000,2.500,Краснодарский край
1248843,11685634611,7378885163,352399695785458,2021-02-03 20:01:01,4607031100011,51.000000,4.000,Республика Хакасия
802190,11730759337,7404185245,352402951220721,2021-02-06 15:26:00,4601653033276,105.000000,1.000,Рязанская область
290780,11714092766,7394966065,352406559653493,2021-02-05 14:33:52,4607087930181,7.200000,30.000,Ивановская область


Размерность датасета:  (2246094, 8)


Описание полей в датасете:  
item_id - Id транзакции  
receipt_id - Id чека  
device_id - Id кассового аппарата  
local_date - Дата и время покупки  
barcode - Штрихкод  
price - Цена покупки, рубли  
quantity - Количество купленного товара, шт.  
region - Регион

Проверим произвольно выбранные штихкоды на сайтах:  
https://ean-online.ru/  и https://service-online.su/text/shtrih-kod/  
4606419005870 - соответствует товару "Сливочное масло Экомилк традиционное 82,5 % 180 г бзмж"  
4607031100011 - соответствует товару "Пиво абаканское аян 0.5л 4.8% ст/б"  
Т.о. наш датасет содержит информацию о покупках различных продуктов, и нам необходимо выбрать только те записи, которые соответствуют покупке напитка Coca-Cola.

После небольшого исследования, приходим к выводу, что штрих код 5449000000286 соответствует товару "Напиток coca-cola безалкогольн газ 2л пэт", который является наиболее покупаемым и содержится в наибольшем количестве (найденных) записей датасета.

Так же были найдены и проверены штрихкоды:  
* 5740700995774 - Напиток coca-cola черри сильногаз жб 330мл - 0 записей
* 5449000017673 - Напиток coca-cola сильногаз 0.5л пл/б - 0 записей
* 54491472 - Напиток газированный coca-cola, 0.5л пластиковая бутылка - 489 записей
* 5449000054227 - Напиток coca-cola безалкогольн газ 1л пэт - 101 запись
* 54490147 - Напиток coca-cola 1l - 0 записей
* 5449000000996 - Напиток газированный coca-cola, 0.33л, жб - 526 записей
* 5449000009067 - Напиток coca-cola 2л - 0 записей
* 5449000000286 - Напиток coca-cola безалкогольн газ 2л пэт - 712 записей
* 5449000131836 - Напиток coca-cola zero безалкогольн газ 0.5л пэт - 22 записи

In [67]:
# Формируем датасет, содержащий информацию только о покупках Кока-колы (2л в пэт упаковке)
mask = data.barcode == '5449000000286' 
df_cola = data[mask]
df_cola.reset_index(drop=True, inplace=True)

### Проведем разведывательный анализ данных

In [68]:
# Выведем 10 случайных строк датафрейма, для предварительного осмотра данных 
display(df_cola.sample(10))
print('Размерность датасета: ', df_cola.shape)

,item_id,receipt_id,device_id,local_date,barcode,price,quantity,region
3,11647074683,7356692611,352406370923721,2021-02-01 00:49:21,5449000000286,120.0,1.0,Санкт-Петербург
243,11706251164,7390623141,352407154193807,2021-02-05 12:26:32,5449000000286,132.0,1.0,Новосибирская область
387,11716327088,7396210880,352405726517704,2021-02-05 16:18:18,5449000000286,130.0,1.0,Санкт-Петербург
233,11727444752,7402338740,352407168358155,2021-02-06 13:49:07,5449000000286,110.0,1.0,Самарская область
46,11675426296,7373109358,352400141191106,2021-02-03 09:33:47,5449000000286,165.0,1.0,Республика Саха (Якутия)
660,11674953621,7372833028,352400118886122,2021-02-02 20:48:09,5449000000286,145.0,1.0,Волгоградская область
174,11725616926,7401327518,352405482118870,2021-02-06 11:31:43,5449000000286,95.0,1.0,Костромская область
422,11686236046,7379233694,352402403010432,2021-02-03 16:34:09,5449000000286,100.0,1.0,Краснодарский край
197,11726201413,7401650040,352405762079371,2021-02-06 17:56:28,5449000000286,150.0,4.0,Республика Саха (Якутия)
135,11690577435,7381730700,352407587297098,2021-02-04 07:01:32,5449000000286,125.0,1.0,Владимирская область


Размерность датасета:  (712, 8)


In [69]:
df_cola.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   item_id     712 non-null    int64  
 1   receipt_id  712 non-null    int64  
 2   device_id   712 non-null    int64  
 3   local_date  712 non-null    object 
 4   barcode     712 non-null    object 
 5   price       712 non-null    float64
 6   quantity    712 non-null    float64
 7   region      712 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 44.6+ KB


In [70]:
display(df_cola.describe())
display(df_cola.describe(include='object'))

,item_id,receipt_id,device_id,price,quantity
count,7.120000e+02,7.120000e+02,7.120000e+02,712.000000,712.000000
mean,1.169810e+10,7.385838e+09,3.524574e+14,123.423455,1.155899
std,2.688573e+07,1.520228e+07,4.744213e+11,21.740303,0.558397
min,1.164707e+10,7.356689e+09,3.523991e+14,16.000000,1.000000
25%,1.167485e+10,7.372771e+09,3.524013e+14,110.000000,1.000000
50%,1.170241e+10,7.388417e+09,3.524041e+14,120.000000,1.000000
75%,1.172078e+10,7.398659e+09,3.524064e+14,135.000000,1.000000
max,1.173505e+10,7.406566e+09,3.566535e+14,200.000000,7.000000


,local_date,barcode,region
count,712,712,712
unique,712,1,53
top,2021-02-02 00:16:27,5449000000286,Республика Саха (Якутия)
freq,1,712,104


Записи датасета не содержат пропуски, а числовые признаки "price" и "quantity" не имеют отрицательных значений и выглядят правдоподобно. 
Датасет содержит данные о покупках из 53 регионов.  
Признак "item_id" (id транзакции) содержит только уникальные значения. При этом, он не является для нас информативным, его можно удалить.

In [71]:
# Удаляем признак "item_id"
df_cola.drop(['item_id'], axis=1, inplace=True)

Проверим записи на дубликаты

In [72]:
print('Количество дублирующихся записей: ', data.duplicated().sum())

Количество дублирующихся записей:  0


Приведем признак "local_date" к типу datetime

In [73]:
df_cola.local_date = pd.to_datetime(df_cola.local_date).dt.to_period('H')
df_cola.head()

,receipt_id,device_id,local_date,barcode,price,quantity,region
0,7364791809,352402250963288,2021-02-02 00:00,5449000000286,140.0,1.0,Нижегородская область
1,7356688842,352402250963288,2021-02-01 00:00,5449000000286,140.0,1.0,Нижегородская область
2,7364799989,352402250963288,2021-02-02 00:00,5449000000286,140.0,1.0,Нижегородская область
3,7356692611,352406370923721,2021-02-01 00:00,5449000000286,120.0,1.0,Санкт-Петербург
4,7377491747,352399667615722,2021-02-03 13:00,5449000000286,110.0,1.0,Московская область


In [74]:
print('Самая ранняя дата в датасете:', df_cola.local_date.min())
print('Самая поздняя дата в датасете:', df_cola.local_date.max())

Самая ранняя дата в датасете: 2021-02-01 00:00
Самая поздняя дата в датасете: 2021-02-07 00:00


Теперь можно удалить признаки receipt_id, device_id, barcode.  
Так же удаляем признак price, - его можно было бы использовать для нахождения цены за единицу товара, но сейчас в этом нет необходимости.

In [75]:
df_cola.drop(['receipt_id', 'device_id', 'barcode', 'price'], axis=1, inplace=True)
df_cola.head()

,local_date,quantity,region
0,2021-02-02 00:00,1.0,Нижегородская область
1,2021-02-01 00:00,1.0,Нижегородская область
2,2021-02-02 00:00,1.0,Нижегородская область
3,2021-02-01 00:00,1.0,Санкт-Петербург
4,2021-02-03 13:00,1.0,Московская область


Т.к. наш датасет содержит только информацию о покупках в конкретные периоды времени, то нам необходимо добавить недостающие периоды, указав для них quantity = 0. 

In [76]:
# Выведем информацию из датафрейма по произвольному региону, для его проверки 
# до и после добавления недостающих отрезков времени
mask1 = df_cola.region == 'Алтайский край' 
df_cola[mask1]

,local_date,quantity,region
94,2021-02-01 10:00,1.0,Алтайский край


In [77]:
list_region = df_cola.region.unique()
# Цикл по регионам
for i in list_region:
    mask = df_cola.region == i
    df_region = df_cola[mask]
    # Считаем количество дней, которые содержат записи о покупках
    delta=(df_region.local_date.max()-df_region.local_date.min()).delta.days+1
    first_day = str(df_cola.local_date.min()).split()[0]
    # Создаем временные отрезки (на каждый час) 
    fh = ForecastingHorizon(
        pd.PeriodIndex(pd.date_range(first_day, periods=delta*24, freq="h")), 
        is_relative=False
        ).to_pandas()
    # Создаем датафрейм, который содежрит все возможные периоды 
    df_tmp = pd.DataFrame()
    df_tmp['local_date'] = fh
    df_tmp['quantity'] = 0
    df_tmp['region'] = i
    # Объединяем и сразу же группируем исходный и созданный датасеты
    df_cola=pd.concat([df_cola, df_tmp], axis=0, join='outer') # по вертикали   
    df_cola = df_cola.groupby(by=['region', 'local_date'], as_index=False).agg({'quantity':'sum'})

# Выводим часть датасета после преобразования (для проверки)    
mask1 = df_cola.region == 'Алтайский край' 
df_cola[mask1]

,region,local_date,quantity
0,Алтайский край,2021-02-01 00:00,0.0
1,Алтайский край,2021-02-01 01:00,0.0
2,Алтайский край,2021-02-01 02:00,0.0
3,Алтайский край,2021-02-01 03:00,0.0
4,Алтайский край,2021-02-01 04:00,0.0
5,Алтайский край,2021-02-01 05:00,0.0
6,Алтайский край,2021-02-01 06:00,0.0
7,Алтайский край,2021-02-01 07:00,0.0
8,Алтайский край,2021-02-01 08:00,0.0
9,Алтайский край,2021-02-01 09:00,0.0


### Обучение модели

Напишем функцию, которая будет на входе принимать название региона, обучать модель и выдавать результат - прогноз покупки количества бутылок Кока-колы по часам в этом регионе

In [78]:
def func_pred(region):
    mask = df_cola.region == region
    df = df_cola[mask]

    # Формируем временной ряд, quantity - целевая переменная
    df = df[['local_date', 'quantity']]
    df = df.set_index('local_date') 

    # Выбираем модель для прогнозирования временных рядов,
    # которая автоматически подберет параметры
    forecaster = AutoARIMA(sp=1, suppress_warnings=True) # sp=1 отсутствие сезонности

    # Если объем выборки позволяет, то разделим её на обучающую и тестовую, 
    # для оценки качества модели
    if (len(df) > 24 ):
        X = df[:len(df)-24] # но лучше делить в соотношении 80 на 20 %
        y = df[len(df)-24:len(df)]
        # Обучение: 
        forecaster.fit(X) 
        # Предсказание:
        y_pred = forecaster.predict(fh=[i for i in range(1,25)]) # fh - количество шагов для прогнозирования        
        print('Метрики для оценки работы модели:')
        # Рассчитываем MAE
        print('MAE score: {:.3f}'.format(metrics.mean_absolute_error(y, y_pred)))
        # Рассчитываем MAPE
        print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y, y_pred) * 100))
        # Рассчитываем RMSE
        print('RMSE score: {:.3f}'.format(np.sqrt(metrics.mean_squared_error(y, y_pred))))    
        # Рассчитываем коэффициент детерминации
        print('R2 score: {:.3f}'.format(metrics.r2_score(y, y_pred)))

    forecaster.fit(df) # обучение модели на всей выборке    
    y_pred = forecaster.predict(fh=[i for i in range(1,25)]) 
    print("Прогноз модели по объему продаж на каждый час:\n", round(y_pred, 0))  

### Прогнозирование

Для получения предсказания выбираем один из регионов из списка ниже и передаем в функцию func_pred()

In [79]:
list(df_cola.region.unique())

['Алтайский край',
 'Астраханская область',
 'Белгородская область',
 'Брянская область',
 'Владимирская область',
 'Волгоградская область',
 'Воронежская область',
 'Иркутская область',
 'Кабардино-Балкарская Республика',
 'Калининградская область',
 'Калужская область',
 'Костромская область',
 'Краснодарский край',
 'Красноярский край',
 'Курская область',
 'Ленинградская область',
 'Москва',
 'Московская область',
 'Мурманская область',
 'Нижегородская область',
 'Новосибирская область',
 'Омская область',
 'Оренбургская область',
 'Орловская область',
 'Пермский край',
 'Приморский край',
 'Республика Бурятия',
 'Республика Коми',
 'Республика Крым',
 'Республика Марий Эл',
 'Республика Саха (Якутия)',
 'Республика Тыва',
 'Ростовская область',
 'Рязанская область',
 'Самарская область',
 'Санкт-Петербург',
 'Саратовская область',
 'Сахалинская область',
 'Свердловская область',
 'Севастополь',
 'Смоленская область',
 'Ставропольский край',
 'Тверская область',
 'Томская область',

In [80]:
# Выводим для наглядности
df_cola[mask1] # Алтайский край

,region,local_date,quantity
0,Алтайский край,2021-02-01 00:00,0.0
1,Алтайский край,2021-02-01 01:00,0.0
2,Алтайский край,2021-02-01 02:00,0.0
3,Алтайский край,2021-02-01 03:00,0.0
4,Алтайский край,2021-02-01 04:00,0.0
5,Алтайский край,2021-02-01 05:00,0.0
6,Алтайский край,2021-02-01 06:00,0.0
7,Алтайский край,2021-02-01 07:00,0.0
8,Алтайский край,2021-02-01 08:00,0.0
9,Алтайский край,2021-02-01 09:00,0.0


In [81]:
func_pred('Алтайский край')

Прогноз модели по объему продаж на каждый час:
                   quantity
2021-02-02 00:00       0.0
2021-02-02 01:00       0.0
2021-02-02 02:00       0.0
2021-02-02 03:00       0.0
2021-02-02 04:00       0.0
2021-02-02 05:00       0.0
2021-02-02 06:00       0.0
2021-02-02 07:00       0.0
2021-02-02 08:00       0.0
2021-02-02 09:00       0.0
2021-02-02 10:00       0.0
2021-02-02 11:00       0.0
2021-02-02 12:00       0.0
2021-02-02 13:00       0.0
2021-02-02 14:00       0.0
2021-02-02 15:00       0.0
2021-02-02 16:00       0.0
2021-02-02 17:00       0.0
2021-02-02 18:00       0.0
2021-02-02 19:00       0.0
2021-02-02 20:00       0.0
2021-02-02 21:00       0.0
2021-02-02 22:00       0.0
2021-02-02 23:00       0.0


In [82]:
func_pred('Московская область')

Метрики для оценки работы модели:
MAE score: 0.586
MAPE score: 91936406089455008.000 %
RMSE score: 1.151
R2 score: 0.243
Прогноз модели по объему продаж на каждый час:
                   quantity
2021-02-07 00:00       1.0
2021-02-07 01:00       1.0
2021-02-07 02:00       1.0
2021-02-07 03:00       1.0
2021-02-07 04:00       1.0
2021-02-07 05:00       1.0
2021-02-07 06:00       1.0
2021-02-07 07:00       1.0
2021-02-07 08:00       1.0
2021-02-07 09:00       1.0
2021-02-07 10:00       1.0
2021-02-07 11:00       1.0
2021-02-07 12:00       1.0
2021-02-07 13:00       1.0
2021-02-07 14:00       1.0
2021-02-07 15:00       1.0
2021-02-07 16:00       1.0
2021-02-07 17:00       1.0
2021-02-07 18:00       1.0
2021-02-07 19:00       1.0
2021-02-07 20:00       1.0
2021-02-07 21:00       1.0
2021-02-07 22:00       1.0
2021-02-07 23:00       1.0


In [83]:
func_pred('Нижегородская область')

Метрики для оценки работы модели:
MAE score: 0.625
MAPE score: 37529996708020808.000 %
RMSE score: 1.058
R2 score: -0.231
Прогноз модели по объему продаж на каждый час:
                   quantity
2021-02-07 00:00       0.0
2021-02-07 01:00       0.0
2021-02-07 02:00       0.0
2021-02-07 03:00       0.0
2021-02-07 04:00       0.0
2021-02-07 05:00       0.0
2021-02-07 06:00       0.0
2021-02-07 07:00       0.0
2021-02-07 08:00       0.0
2021-02-07 09:00       0.0
2021-02-07 10:00       0.0
2021-02-07 11:00       0.0
2021-02-07 12:00       0.0
2021-02-07 13:00       0.0
2021-02-07 14:00       0.0
2021-02-07 15:00       0.0
2021-02-07 16:00       0.0
2021-02-07 17:00       0.0
2021-02-07 18:00       0.0
2021-02-07 19:00       0.0
2021-02-07 20:00       0.0
2021-02-07 21:00       0.0
2021-02-07 22:00       0.0
2021-02-07 23:00       0.0


In [84]:
# Выводим для наглядности
mask1 = df_cola.region == 'Краснодарский край'
df_cola[mask1]

,region,local_date,quantity
1139,Краснодарский край,2021-02-01 00:00,0.0
1140,Краснодарский край,2021-02-01 01:00,2.0
1141,Краснодарский край,2021-02-01 02:00,0.0
1142,Краснодарский край,2021-02-01 03:00,0.0
1143,Краснодарский край,2021-02-01 04:00,0.0
...,...,...,...
1278,Краснодарский край,2021-02-06 19:00,4.0
1279,Краснодарский край,2021-02-06 20:00,1.0
1280,Краснодарский край,2021-02-06 21:00,0.0
1281,Краснодарский край,2021-02-06 22:00,0.0


In [85]:
func_pred('Краснодарский край')

Метрики для оценки работы модели:
MAE score: 0.833
MAPE score: 150119987579016608.000 %
RMSE score: 1.118
R2 score: -0.023
Прогноз модели по объему продаж на каждый час:
                   quantity
2021-02-07 00:00      -0.0
2021-02-07 01:00       0.0
2021-02-07 02:00       0.0
2021-02-07 03:00       1.0
2021-02-07 04:00       1.0
2021-02-07 05:00       1.0
2021-02-07 06:00       1.0
2021-02-07 07:00       0.0
2021-02-07 08:00       0.0
2021-02-07 09:00       0.0
2021-02-07 10:00       1.0
2021-02-07 11:00       1.0
2021-02-07 12:00       1.0
2021-02-07 13:00       1.0
2021-02-07 14:00       1.0
2021-02-07 15:00       1.0
2021-02-07 16:00       0.0
2021-02-07 17:00       1.0
2021-02-07 18:00       1.0
2021-02-07 19:00       1.0
2021-02-07 20:00       1.0
2021-02-07 21:00       1.0
2021-02-07 22:00       1.0
2021-02-07 23:00       1.0


Метрика MAE помогает оценить абсолютную ошибку: насколько в среднем число в предсказании разошлось с реальным числом, но в данном случае она неинформативна.  
Стоит обратить внимание на MAPE - таже метрика, только в процентах. Её значение уходит в бесконечность, - это можно интерпретировать так, что происходит деление на ноль (действительно, почти все фактические значения равны нулю).  
R2 - по этой метрике можно оценить качество модели, она измеряется от -бесконечности до 1, а удовлетворительным показателем считается значение выше 0.5. В нашем случае, отрицательное значение метрики говорит о проблеме с моделью прогнозирования и качеством/размером выборки данных.